# 4. Ensembles de Arboles de Decision

Un arbol de decisión es un modelo débil, el aumento del poder predictivo proviene al ensamblar varios arboles de decisión.
<br> Si promedio n arboles identicos, el resultados es exactamente el mismo que utilizar un solo arbol, necesito PERTURBAR cada arbol para disponer de variablidad

la variabilidad provendrá de estas fuentes:


*   Perturbar el dataset
*   Perturbar el algoritmo del arbol
*   Perturbar el dataset y el algoritmo del arbol al mismo tiempo

Se verán estos tres algoritmos


*   Arboles Azarosos
*   Random Forest
*   Gradient Boosting of Decision Trees

#### 4.01 Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi



---



## 4.02 Arboles Azarosos

Arboles Azarosos es el nombre de un algoritmo trivial (por favor NO confundir con Random Forest)
Qué tipo de perturbaciones se realizan en Arboles Azarosos
* Se perturba el dataset
* No se perturba el algoritmo, es siempre rpart original

Cada  arbolito de  Arboles Azarosos se entrena sobre un dataset perturbado,  que tiene exactamente la misma cantidad de registros pero solo un subconjunto de los atributos (campos)  del dataset, tomados al azar, de los originales.
<br> En esta primera corrida, se construira cada arbol en un dataset utilizando el 50% de los campos

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sun Oct 26 11:51:27 PM 2025"

In [2]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,660381,35.3,1454471,77.7,1454471,77.7
Vcells,1226627,9.4,8388608,64.0,1975128,15.1


In [3]:
# cargo las librerias que necesito
require("data.table")
require("rpart")

Loading required package: data.table

Loading required package: rpart



Aqui debe cargar SU semilla primigenia

In [ ]:
PARAM <- list()
PARAM$semilla_primigenia <- 191717

# parametros  arbol
# entreno cada arbol con solo 50% de las variables variables
#  por ahora, es fijo
PARAM$feature_fraction <- 0.5

PARAM$rpart$cp <- -1
PARAM$rpart$minsplit <- 750
PARAM$rpart$minbucket <- 41
PARAM$rpart$maxdepth <- 10

# voy a generar 512 arboles,
#  a mas arboles mas tiempo de proceso y MEJOR MODELO,
#  pero ganancias marginales
PARAM$num_trees_max <- 512

In [13]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento <- "exp420_bo"
dir.create(experimento, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))

In [14]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

In [15]:
# defino los dataset de entrenamiento y aplicacion
dtrain <- dataset[foto_mes == 202107]
dfuture <- dataset[foto_mes == 202109]

# arreglo clase_ternaria por algun distraido ""
dfuture[, clase_ternaria := NA ]

In [16]:
# Establezco cuales son los campos que puedo usar para la prediccion
# el copy() es por la Lazy Evaluation
campos_buenos <- copy(setdiff(colnames(dtrain), c("clase_ternaria")))

In [17]:
# que tamanos de ensemble grabo a disco
grabar <- c(1, 2, 4, 8, 16, 32, 64, 128, 256, 384, 512)

In [18]:
tb_prediccion <- dfuture[, list(numero_de_cliente)]
# aqui se va acumulando la probabilidad del ensemble
tb_prediccion[, prob_acumulada := 0]

In [ ]:
# for (arbolito in seq(PARAM$num_trees_max) ) {
#   message( arbolito, " ")
#   qty_campos_a_utilizar <- as.integer(length(campos_buenos)
#     * PARAM$feature_fraction)

#   # elijo los campos al azar
#   campos_random <- sample(campos_buenos, qty_campos_a_utilizar)

#   # paso de un vector a un string con los elementos
#   # separados por un signo de "+"
#   # este hace falta para la formula
#   campos_random <- paste(campos_random, collapse= " + ")

#   # armo la formula para rpart
#   formulita <- paste0("clase_ternaria ~ ", campos_random)

#   # genero el arbol de decision
#   modelo <- rpart(formulita,
#     data= dtrain,
#     xval= 0,
#     control= PARAM$rpart
#   )

#   # aplico el modelo a los datos que no tienen clase
#   prediccion <- predict(modelo, dfuture, type= "prob")

#   tb_prediccion[, prob_acumulada := prob_acumulada + prediccion[, "BAJA+2"]]

#   if (arbolito %in% grabar) {
#     umbral_corte <- (1 / 40) * arbolito
#     tb_prediccion[, Predicted := as.numeric(prob_acumulada > umbral_corte)]

#     archivo_kaggle <- paste0(
#         "KA420_",
#         sprintf("%.3d", arbolito), # para que tenga ceros adelante
#         ".csv"
#       )

#     # grabo el archivo
#     fwrite( tb_prediccion[, list(numero_de_cliente, Predicted)],
#       file= archivo_kaggle,
#       sep= ","
#     )

#     # subida a Kaggle
#     comando <- "kaggle competitions submit"
#     competencia <- "-c labo-i-2025-virtual-analista-sr"
#     arch <- paste( "-f", archivo_kaggle)

#     mensaje <- paste0("-m 'cp=", PARAM$rpart$cp, "  minsplit=", PARAM$rpart$minsplit, "  minbucket=", PARAM$rpart$minbucket, " maxdepth=", PARAM$rpart$maxdepth, "'" )
#     linea <- paste( comando, competencia, arch, mensaje)
#     salida <- system(linea, intern=TRUE)
#     cat(salida)
#   }
# }


In [20]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Mon Oct 27 01:26:25 AM 2025"

In [27]:
require("data.table")
require("rpart")
require("parallel")

if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")
if( ! require("rlist") ) install.packages("rlist")
require("rlist")
if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")


Loading required package: parallel

Loading required package: DiceKriging

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘DiceKriging’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: DiceKriging

Loading required package: rlist

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘rlist’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘XML’


Loading required package: rlist

Loading required package: mlrMBO

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘mlrMBO’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘fastmatch’, ‘RcppArmadillo’, ‘mlr’, ‘

In [28]:
library(tools) # Para basename()

#' Obtiene la fila superior (más reciente) de la tabla de envíos.
#' Incluye reintentos internos en caso de fallo de la API.
#'
#' @param competition_name El nombre de la competición.
#' @param max_attempts_get Número de reintentos para este comando.
#' @return Una fila de data.table (el envío más reciente) o NULL.
get_most_recent_submission <- function(competition_name, max_attempts_get = 3) {

  submissions_cmd <- sprintf(
    "kaggle competitions submissions -c %s --csv",
    shQuote(competition_name, type = "sh")
  )

  for (i in 1:max_attempts_get) {
    sub_table <- NULL
    csv_output <- NULL

    tryCatch({
      csv_output <- system(submissions_cmd, intern = TRUE, ignore.stderr = TRUE)

      if (length(csv_output) < 2) {
        cat("Aviso: No se encontraron envíos previos.\n")
        return(NULL)
      }

      sub_table <- fread(text = csv_output)

      if (nrow(sub_table) > 0) {
        return(sub_table[1, ]) # Devuelve solo la fila más reciente
      }

    }, error = function(e) {
      cat("Error al obtener/parsear envíos (Intento", i, "):", e$message, "\n")
    })

    if (i < max_attempts_get) {
      Sys.sleep(5)
    }
  }

  cat("Error: Fallo al obtener la lista de envíos después de", max_attempts_get, "intentos.\n")
  return(NULL)
}


#' Sube un archivo a Kaggle y sondea para obtener el score.
#'
#' @param competition_name El nombre 'slug' de la competición.
#' @param file_path La ruta al archivo .csv que se quiere subir.
#' @param message El mensaje de envío (tu `-m` note).
#' @param max_submit_attempts Reintentos para el comando de envío.
#' @param max_poll_attempts Reintentos para el sondeo de score.
#' @param wait_seconds Segundos de espera entre sondeos.
#'
#' @return El 'publicScore' (numérico) si tiene éxito.
#'         Devuelve NULL si falla o se agota el tiempo.
#'
submit_and_get_score <- function(competition_name,
                                 file_path,
                                 message,
                                 max_submit_attempts = 3,
                                 max_poll_attempts = 5,
                                 wait_seconds = 60) {

  # --- 1. Verificación preliminar ---
  if (!file.exists(file_path)) {
    cat("Error: El archivo no existe en la ruta:", file_path, "\n")
    return(NULL)
  }

  submission_filename <- basename(file_path)

  # --- 2. Obtener el estado "Antes del envío" ---
  cat("Obteniendo el estado del último envío (antes de subir)...\n")
  old_submission <- get_most_recent_submission(competition_name)

  old_submission_date <- if (!is.null(old_submission)) old_submission$date else ""
  cat("Fecha del último envío conocido:", old_submission_date, "\n")


  # --- 3. Ejecutar el envío a Kaggle (CON REINTENTOS) ---
  cat("Iniciando envío de:", submission_filename, "a", competition_name, "...\n")

  submit_cmd <- sprintf(
    "kaggle competitions submit -c %s -f %s -m %s -q",
    shQuote(competition_name, type = "sh"),
    shQuote(file_path, type = "sh"),
    shQuote(message, type = "sh")
  )

  submit_success <- FALSE
  for (i in 1:max_submit_attempts) {
    submit_output <- system(submit_cmd, intern = TRUE, ignore.stderr = TRUE)

    if (length(submit_output) > 0 && grepl("Successfully submitted",
        submit_output[1], ignore.case = TRUE)) {
      submit_success <- TRUE
      cat("Envío exitoso. Esperando a Kaggle para procesar...\n")
      break
    } else {
      cat("Fallo en el envío (Intento", i, "). Reintentando en 10 segundos...\n")
      print(submit_output)
      Sys.sleep(10)
    }
  }

  if (!submit_success) {
    cat("Error: El envío falló después de", max_submit_attempts, "intentos.\n")
    return(NULL)
  }

  # --- 4. Bucle de sondeo (Polling) para el resultado ---
  for (i in 1:max_poll_attempts) {
    cat("--- Sondeo", i, "de", max_poll_attempts, "---\n")

    if (i == 1) {
      cat("Esperando", wait_seconds, "segundos iniciales...\n")
    } else {
      cat("Esperando", wait_seconds, "segundos...\n")
    }
    Sys.sleep(wait_seconds)


    new_submission <- get_most_recent_submission(competition_name)

    if (is.null(new_submission)) {
      cat("Aún no aparece ningún envío. Reintentando...\n")
      next
    }

    # --- 5. Lógica de validación ---

    cat("Viendo Fila 1:",
        "File:", new_submission$fileName,
        "| Date:", new_submission$date,
        "| Status:", new_submission$status, "\n")

    # Comprobamos las 3 condiciones
    date_has_changed <- (new_submission$date != old_submission_date)

    # *** CAMBIO REALIZADO AQUÍ ***
    status_is_complete <- (new_submission$status == "SubmissionStatus.COMPLETE")

    filename_matches <- (new_submission$fileName == submission_filename)

    if (date_has_changed && filename_matches && status_is_complete) {
      cat("¡Éxito! Nuevo envío verificado y completado.\n")
      cat("Score:", new_submission$publicScore, "\n")
      return(new_submission$publicScore)

    # *** CAMBIO REALIZADO AQUÍ ***
    } else if (date_has_changed && filename_matches && new_submission$status == "SubmissionStatus.ERROR") {
      cat("Error: El envío ha sido procesado por Kaggle pero ha fallado.\n")
      return(NULL)

    } else {
      cat("El envío aún no está listo o no coincide. (Fecha Cambiada:", date_has_changed,
          "| Nombre Coincide:", filename_matches,
          "| Estado Completo:", status_is_complete, ")\n")
    }
  }

  cat("Error: Se agotó el tiempo de espera. No se pudo obtener el score después de", max_poll_attempts, "intentos.\n")
  return(NULL)
}

In [44]:
PARAM <- list()
PARAM$semilla_primigenia <- 191717

# parametros  arbol
# entreno cada arbol con solo 50% de las variables variables
#  por ahora, es fijo
PARAM$feature_fraction <- 0.5

# voy a generar 32 arboles,
#  a mas arboles mas tiempo de proceso y MEJOR MODELO,
#  pero ganancias marginales
PARAM$num_trees_max <- 32

ArbolesAzarosos <- function(iteracion, x) {

  tb_prediccion <- dfuture[, list(numero_de_cliente)]
  # aqui se va acumulando la probabilidad del ensemble
  tb_prediccion[, prob_acumulada := 0]

  set.seed(PARAM$semilla_primigenia)
  cat("Iteracion: ", iteracion, "\n")
  cat("Params: ", paste0("cp=", x$cp, "  minsplit=", x$minsplit, "  minbucket=", x$minbucket, " maxdepth=", x$maxdepth, "" ), "\n")
  flush.console()

  seq(PARAM$num_trees_max)

  arbolito <- 1

  for (arbolito in seq(PARAM$num_trees_max) ) {
    cat( arbolito, "")
    flush.console()
    qty_campos_a_utilizar <- as.integer(length(campos_buenos)
      * PARAM$feature_fraction)

    # elijo los campos al azar
    campos_random <- sample(campos_buenos, qty_campos_a_utilizar)

    # paso de un vector a un string con los elementos
    # separados por un signo de "+"
    # este hace falta para la formula
    campos_random <- paste(campos_random, collapse= " + ")

    # armo la formula para rpart
    formulita <- paste0("clase_ternaria ~ ", campos_random)

    # genero el arbol de decision
    modelo <- rpart(formulita,
      data= dtrain,
      xval= 0,
      control= x
    )

    # aplico el modelo a los datos que no tienen clase
    prediccion <- predict(modelo, dfuture, type= "prob")

    tb_prediccion[, prob_acumulada := prob_acumulada + prediccion[, "BAJA+2"]]

  }
  cat("finalizo construccion de arbolitos\n")
  flush.console()

  # reporto ganancia
  umbral_corte <- (1 / 40) * PARAM$num_trees_max
  tb_prediccion[, Predicted := as.numeric(prob_acumulada > umbral_corte)]

  archivo_kaggle <- paste0(
    "KA420_BO_0_",
    sprintf("%.3d", iteracion), # para que tenga ceros adelante
    ".csv"
  )

  fwrite( tb_prediccion[, list(numero_de_cliente, Predicted)],
    file= archivo_kaggle,
    sep= ","
  )

  competition_name  <- "labo-i-2025-virtual-analista-sr"
  mensaje <- paste0("cp=", x$cp, "  minsplit=", x$minsplit, "  minbucket=", x$minbucket, " maxdepth=", x$maxdepth, "" )
  score <- submit_and_get_score(competition_name,
                        archivo_kaggle,
                        mensaje,
                        max_submit_attempts = 5,
                        max_poll_attempts = 5,
                        wait_seconds = 60)
  cat("score reportado: ", score, "\n")
  flush.console()
  score
}

In [10]:
  # comando <- "kaggle competitions submissions"
  # competencia <- "-c labo-i-2025-virtual-analista-sr"

  # linea <- paste( comando, competencia, "--csv")
  # salida <- system(linea, intern=TRUE)
  # subs <- fread(text = salida)
  # subs[1]

fileName,date,description,status,publicScore,privateScore
<chr>,<dttm>,<chr>,<chr>,<dbl>,<lgl>
KA420_256.csv,2025-10-26 22:53:08,cp=-1 minsplit=475 minbucket=238 maxdepth=7,SubmissionStatus.COMPLETE,54.727,NA


In [39]:
EstimarGanancia <- function(x) {
  message(format(Sys.time(), "%a %b %d %X %Y"))
  GLOBAL_iteracion <<- GLOBAL_iteracion + 1

  # param= x los hiperparametros del arbol
  # qfolds= xval_folds  la cantidad de folds
  ganancia <- ArbolesAzarosos(GLOBAL_iteracion, x)

  return(ganancia)
}


In [45]:
# Defino la  Optimizacion Bayesiana

# cantidad de iteraciones de la Optimizacion Bayesiana
PARAM <- list()

PARAM$semilla_primigenia <- 191717
#PARAM$experimento <- "HT420"

PARAM$BO_iter <- 40 #cantidad de iteraciones de la Bayesian Optimization

# la letra L al final de 1L significa ENTERO
PARAM$hs <- makeParamSet(
  makeNumericParam("cp", lower= -1, upper= 0.1),
  makeIntegerParam("minsplit", lower= 1L, upper= 8000L),
  makeIntegerParam("minbucket", lower= 1L, upper= 4000L),
  makeIntegerParam("maxdepth", lower= 3L, upper= 20L),
  forbidden= quote(minbucket > 0.5 * minsplit)
)
# minbuket NO PUEDE ser mayor que la mitad de minsplit

archivo_log <- "BO_log.txt"
archivo_BO <- "bayesian.RDATA"

# leo si ya existe el log
#  para retomar en caso que se se corte el programa
GLOBAL_iteracion <- 0
GLOBAL_mejor <- -Inf

if (file.exists(archivo_log)) {
  tabla_log <- fread(archivo_log)
  GLOBAL_iteracion <- nrow(tabla_log)
  GLOBAL_mejor <- tabla_log[, max(ganancia)]
}

# Aqui comienza la configuracion de la Bayesian Optimization

funcion_optimizar <- EstimarGanancia

configureMlr(show.learner.output= FALSE)

# configuro la busqueda bayesiana,
#  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
# minimize= FALSE estoy Maximizando la ganancia
obj.fun <- makeSingleObjectiveFunction(
  fn= funcion_optimizar,
  minimize= FALSE,
  noisy= TRUE,
  par.set= PARAM$hs,
  has.simple.signature= FALSE
)

ctrl <- makeMBOControl(
  save.on.disk.at.time= 600,
  save.file.path= archivo_BO
)

ctrl <- setMBOControlTermination(ctrl, iters= PARAM$BO_iter)
ctrl <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

surr.km <- makeLearner("regr.km",
                       predict.type= "se",
                       covtype= "matern3_2", control= list(trace= TRUE)
)


# inicio la optimizacion bayesiana
if (!file.exists(archivo_BO)) {
  bayesiana_salida <- mbo(
    fun= obj.fun,
    learner= surr.km,
    control= ctrl
  )
} else {
  bayesiana_salida <- mboContinue(archivo_BO)
}
# retomo en caso que ya exista


# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
        file= "BO_log.txt",
        sep= "\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  list(cp, minsplit, minbucket, maxdepth)
]

print(PARAM$out$lgbm$mejores_hiperparametros)

Computing y column(s) for design. Not provided.

Mon Oct 27 01:53:35 AM 2025



Iteracion:  1 
Params:  cp=-0.190153969467792  minsplit=6173  minbucket=430 maxdepth=5 
finalizo construccion de arbolitos




---

